In [7]:
# Class: COEN 140 Lab Machine Learning/Data Mining
# Name: Matthew Davenport
# Date: 9/25/2022
# Title: Lab 8 – K-Means Clustering

import numpy as np
import math

Importing Data 

In [8]:
# import data from txt file for test data and training data
import pandas as pd

train_data = pd.read_csv('C:/Users/daven/COEN140/Lab7/spambase/spam-train', header=None)
test_data = pd.read_csv('C:/Users/daven/COEN140/Lab7/spambase/spam-test', header=None)

print(train_data)
print(test_data)

        0     1     2    3     4     5    6    7     8     9   ...     48  \
0     0.00  0.00  0.00  0.0  1.01  0.00  0.0  0.0  0.00  0.00  ...  0.000   
1     0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.00  ...  0.098   
2     0.00  0.53  0.00  0.0  1.06  0.00  1.6  0.0  0.00  0.53  ...  0.000   
3     0.00  0.00  0.23  0.0  0.92  0.00  0.0  0.0  0.23  0.00  ...  0.000   
4     0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.00  ...  0.000   
...    ...   ...   ...  ...   ...   ...  ...  ...   ...   ...  ...    ...   
3060  0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.00  ...  0.000   
3061  0.00  0.00  0.00  0.0  1.96  0.00  0.0  0.0  0.00  0.00  ...  0.000   
3062  0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.00  ...  0.000   
3063  0.33  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.00  ...  0.000   
3064  0.31  0.00  0.62  0.0  0.00  0.31  0.0  0.0  0.00  0.00  ...  0.000   

         49     50     51     52     53     54    55     56   57  
0     0.

Splitting training and testing data sets into X and y sets

In [9]:
X_train = train_data.drop(57, axis=1)
y_train = train_data[57]
X_test = test_data.drop(57, axis=1)
y_test = test_data[57]

print(X_test.shape)
print(y_test.shape)

(1536, 57)
(1536,)


In [10]:
# standardizing function: standardize data by subtracting the mean and dividing by standard deviation
def standardize(data):
    return (data - data.mean(axis=0)) / data.std(axis=0)

# sigmoid function to compute sigmoid nescessary for logistic regression
def sigmoid(r):
    return 1 / (1 + math.exp(-r))

# need sigmoid function to be vectorized 
vectorized_sigmoid = np.vectorize(sigmoid)

#standardize training and testing datasets
X_train = standardize(X_train)
X_test = standardize(X_test)

# add dummy feature for X_train and X_test datasets
X_training_dummy = X_train.copy()
X_training_dummy['dummy'] = np.ones(X_training_dummy.shape[0])
X_testing_dummy = X_test.copy()
X_testing_dummy['dummy'] = np.ones(X_testing_dummy.shape[0])

print(X_testing_dummy.shape)
 
# initialize w to vector of zeros with proper shape
w_init = np.random.normal(0,1,size = X_testing_dummy.shape[1])
print(w_init.shape)

alpha = 5 * (10e-5)
threshold = 10e-5

(1536, 58)
(58,)


In [11]:
# optimize model using gradient descent
def model_optimize(w, X, y):
    e = 1
    while e > threshold: 
        w_temp = w - alpha * (np.dot((vectorized_sigmoid(np.dot(X,w)) - y), X))
        e = np.linalg.norm(w_temp - w)
        w = w_temp
        
    return w

# predict based on our sigmoid function
def model_predict(w, X):
    return vectorized_sigmoid(np.dot(w, X.T))

# classify our predictions to a 0 or 1, spam or not 
def model_binary_predict(predictions):
    binary_predictions = []
    for x in predictions:
        if x < 0.5:
            binary_predictions.append(0)
        else:
            binary_predictions.append(1)
            
    return binary_predictions

#calulcate error
def error(actuals, predictions):
    correct_predictions = 0
    total = len(y_test)
    for i in range(total):
        if predictions[i] == actuals[i]:
            correct_predictions += 1
    correct = (correct_predictions / total) * 100
    return 100 - correct



In [12]:
w_train = model_optimize(w_init, X_training_dummy, y_train)
w_test = model_optimize(w_init, X_testing_dummy, y_test)

predictions_training = model_binary_predict(model_predict(w_train, X_training_dummy))
predictions_testing = model_binary_predict(model_predict(w_test, X_testing_dummy))

error_training = error(y_train, predictions_training)
error_testing = error(y_test, predictions_testing)

print("Training error: ", error_training, "%")
print("Testing error: ", error_testing, "%")



Training error:  6.119791666666657 %
Testing error:  6.0546875 %
